In [1]:
# Download Feature Enginering Data
#!wget --header="Host: doc-0s-0c-docs.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9,hi;q=0.8" --header="Cookie: AUTH_g6rouv7anedfrrf7ue0ns472lo11dqgi=03254098180734767806|1615005225000|erg5lfu672cth1uovkajhnpo83i0rk8k" --header="Connection: keep-alive" "https://doc-0s-0c-docs.googleusercontent.com/docs/securesc/pblgkejb48lv4je87rr0ale25k6q65pa/lsqmpv4sus5b9qehbisah0rh7p26ek9b/1615005225000/03254098180734767806/03254098180734767806/1eOWhua4r1ZVzvP1yuTnPXi_RbOcrbmjf?e=download&authuser=0" -c -O 'Data_m5.pkl'


In [2]:
# Load All The Libraries

import pandas as pd
import numpy as np
import random
from  datetime import datetime, timedelta
import joblib

In [3]:
# Read The Data

data = pd.read_pickle("Data_m5.pkl")

In [4]:
data.shape

(27023821, 47)

**1. Choose Any Item ID Randomly**

In [5]:
m1 = random.randrange(data.shape[0])
unique_id = data['id'][m1]
unique_id

'FOODS_2_137_TX_3_evaluation'

In [6]:
data_point = data[data['id']==unique_id]
data_point

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_type_1,event_type_2,snap_CA,snap_TX,snap_WI,event_name_1_Cinco De Mayo,event_name_1_Father's day,event_name_1_MemorialDay,event_name_1_Mother's day,event_name_1_NBAFinalsEnd,event_name_1_NBAFinalsStart,event_name_1_OrthodoxEaster,event_name_1_Pesach End,event_name_1_Ramadan starts,event_name_2_Cinco De Mayo,event_name_2_Father's day,event_name_2_OrthodoxEaster,Event_1,Event_2,group_day,sell_price,price_diffrence_2,price_diffrence_4,price_diffrence_8,lag_7,lag_28,rmean_7_7,rmean_7_28,rmean_28_7,rmean_28_28,week,quarter,mday
16995,FOODS_2_137_TX_3_evaluation,1887,5,2,6,1,d_1050,0.0,2013-12-13,11346,0,7,12,2013,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,3,4.46875,-0.0,-0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,50,4,13
144994,FOODS_2_137_TX_3_evaluation,1887,5,2,6,1,d_1051,0.0,2013-12-14,11347,2,1,12,2013,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,3,4.46875,-0.0,-0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,50,4,14
144995,FOODS_2_137_TX_3_evaluation,1887,5,2,6,1,d_1052,0.0,2013-12-15,11347,3,2,12,2013,0,0,0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,3,4.46875,-0.0,-0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,50,4,15
144996,FOODS_2_137_TX_3_evaluation,1887,5,2,6,1,d_1053,0.0,2013-12-16,11347,1,3,12,2013,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,4,4.46875,-0.0,-0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,51,4,16
144997,FOODS_2_137_TX_3_evaluation,1887,5,2,6,1,d_1054,0.0,2013-12-17,11347,5,4,12,2013,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,4,4.46875,-0.0,-0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,51,4,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26891214,FOODS_2_137_TX_3_evaluation,1887,5,2,6,1,d_1965,NaN,2016-06-15,11620,6,5,6,2016,0,0,0,1,1,0.0,0.0,0.0,0.0,27.0,0.0,0.0,0.0,0.0,0.0,27.0,0.0,0,0,3,4.46875,-0.0,-0.0,0.0,NaN,0.0,NaN,NaN,0.856934,1.036133,24,2,15
26891215,FOODS_2_137_TX_3_evaluation,1887,5,2,6,1,d_1966,NaN,2016-06-16,11620,4,6,6,2016,0,0,0,0,0,0.0,0.0,0.0,0.0,28.0,0.0,0.0,0.0,0.0,0.0,28.0,0.0,0,0,4,4.46875,-0.0,-0.0,0.0,NaN,0.0,NaN,NaN,0.856934,0.893066,24,2,16
26891216,FOODS_2_137_TX_3_evaluation,1887,5,2,6,1,d_1967,NaN,2016-06-17,11620,0,7,6,2016,0,0,0,0,0,0.0,0.0,0.0,0.0,29.0,0.0,0.0,0.0,0.0,0.0,29.0,0.0,0,0,4,4.46875,-0.0,-0.0,0.0,NaN,1.0,NaN,NaN,1.000000,0.893066,24,2,17
27003355,FOODS_2_137_TX_3_evaluation,1887,5,2,6,1,d_1968,NaN,2016-06-18,11621,2,1,6,2016,0,0,0,0,0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0,0,4,4.46875,-0.0,-0.0,0.0,NaN,0.0,NaN,NaN,0.428467,0.893066,24,2,18


**2. Downcast The Data**

In [7]:
#https://www.kaggle.com/anshuls235/time-series-forecasting-eda-fe-modelling/notebook#2.-Downcasting

def downcast(df):

    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()

    for i,t in enumerate(types):

        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)

        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)

        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')

    return df  

In [8]:
data = downcast(data)

**3. Preprocessing The Data**

In [9]:
def preprocessing(data):
    """
     to preprocees the data 
    """

    # A.  Get integer value in d column    ex d_1 , d_2   ---->>>   1  ,1
    data['d'] = data['d'].apply(lambda a: a.split('_')[1]).astype(np.int16)

    # B. Filter The Data
    dt = data[data['d']>=1871]
    return dt

**4. Feature Engineering The Data**

In [10]:
def feature_engineering(dt):
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)

    wins = [7, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())

    return dt

In [11]:
h = 28 
max_lags = 70
fday = datetime(2016,5, 23) 
fday

datetime.datetime(2016, 5, 23, 0, 0)

In [12]:
def final_Pred1(data_point):

  dt = preprocessing(data_point)

  
  useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday", "sell_price",'event_name_1', 'event_name_2','Event_1','Event_2']
  train_cols = dt.columns[~dt.columns.isin(useless_cols)]

  # Load The Model
  m_lgb = joblib.load("lgb_Model_0.pkl")
  for tdelta in range(0, 28):
      day = fday + timedelta(days=tdelta)
      print(day)
      tst = dt[(dt.date >= day - timedelta(days=max_lags)) & (dt.date <= day)]
      tst = feature_engineering(tst)
      tst = tst.loc[tst.date == day , train_cols]
      dt.loc[dt.date == day, "sales"] = m_lgb.predict(tst)
      print("Prediction Compltete ",tdelta)
      del(tst)

  x = pd.pivot_table(dt, index='id', values='sales', columns='d').iloc[:,-28:].reset_index()

  return x

In [13]:
dt1 = final_Pred1(data_point)
dt1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


2016-05-23 00:00:00


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

Prediction Compltete  0
2016-05-24 00:00:00
Prediction Compltete  1
2016-05-25 00:00:00
Prediction Compltete  2
2016-05-26 00:00:00
Prediction Compltete  3
2016-05-27 00:00:00
Prediction Compltete  4
2016-05-28 00:00:00
Prediction Compltete  5
2016-05-29 00:00:00
Prediction Compltete  6
2016-05-30 00:00:00
Prediction Compltete  7
2016-05-31 00:00:00
Prediction Compltete  8
2016-06-01 00:00:00
Prediction Compltete  9
2016-06-02 00:00:00
Prediction Compltete  10
2016-06-03 00:00:00
Prediction Compltete  11
2016-06-04 00:00:00
Prediction Compltete  12
2016-06-05 00:00:00
Prediction Compltete  13
2016-06-06 00:00:00
Prediction Compltete  14
2016-06-07 00:00:00
Prediction Compltete  15
2016-06-08 00:00:00
Prediction Compltete  16
2016-06-09 00:00:00
Prediction Compltete  17
2016-06-10 00:00:00
Prediction Compltete  18
2016-06-11 00:00:00
Prediction Compltete  19
2016-06-12 00:00:00
Prediction Compltete  20
2016-06-13 00:00:00
Prediction Compltete  21
2016-06-14 00:00:00
Prediction Compltete

d,id,1942,1943,1944,1945,1946,1947,1948,1949,1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969
0,FOODS_2_137_TX_3_evaluation,1.094727,0.977051,0.92334,0.868652,0.868164,1.066406,1.251953,0.995117,0.88916,1.201172,1.052734,1.321289,1.192383,1.331055,1.321289,1.124023,1.053711,1.117188,0.991211,1.287109,1.352539,1.111328,0.98584,1.052734,0.87793,0.920898,1.058594,1.061523
